# Credit-Migration-Analysis-with-SQL

This project aims at demonstrating how to generate a Migration Analysis (also called Roll Rate Analysis) for a loan portfolio between two points in time.
This analysis is useful for forecasting default volumes and and also for determining the definition of bad loans.

In [1]:
# importing the needed libraries
import pandas as pd
import duckdb

### Loading the data and registering the dataframes in DuckDB

In [2]:
# initialise duckdb connection
con = duckdb.connect()

# Loading tables with pandas
migration_base_df = pd.read_csv('Anon_MigrationBase.csv', sep=';')
processed_data_df = pd.read_csv('Anon_Processed_Data.csv', sep=';')
woff_data_df = pd.read_csv('Anon_Woff_Data.csv', sep=';')

# Register tabels in DuckDB
con.register('MigrationBase', migration_base_df)
con.register('Processed_Data', processed_data_df)
con.register('Woff_Data', woff_data_df)

### 1. Generates a breakdown of loan amounts by arrears bucket in Reference Period

In [9]:
#Step 1: Migration_BaseBuckets

# Delete table if already exists
con.execute("DROP TABLE IF EXISTS Migration_BaseBuckets;")

# Generate Arrears bucket for first period
con.execute("""
    CREATE TABLE Migration_BaseBuckets AS
    SELECT 
        CategoryBase AS Arrears_Category,
        SUM(COALESCE(OS_Principal_LC, 0)) AS OS_Principal
    FROM MigrationBase
    GROUP BY CategoryBase
    ORDER BY CategoryBase;
""")

#check the outputs
con.execute("SELECT * FROM Migration_BaseBuckets").df()

,Arrears_Category,OS_Principal
0,A_Current,30813690.80
1,B_1_to_30_days,71180.90
2,C_31_to_60_days,9492.70
3,D_61_to_90_days,142050.73
4,E_91_to_120_days,35354.34
5,F_121_to_180_days,197256.36
6,G_181_to_270_days,56960.26


### 2. Crosstabulation of outstanding loan amounts by arrears categories in the two periods

In [10]:
#Step 2: Migration_NewBuckets (Cross-tab)

# Delete table if already exists
con.execute("DROP TABLE IF EXISTS Migration_NewBuckets;")

# The query links the two portfolio data and generates a cross tabulation of outstanding loan amounts 
# in the new data where the rows are the old arrears buckets and the columns are the new ones.
# Left join is used because I want only the loans that exist in two periods (i.e. exclude new loans)
con.execute("""
    CREATE TABLE Migration_NewBuckets AS
    SELECT 
        mb.CategoryBase AS Arrears_Category,
        SUM(CASE WHEN pd.Arrears_Category = 'A_Current' THEN pd.OS_Principal_LC ELSE 0 END) AS A_Current,
        SUM(CASE WHEN pd.Arrears_Category = 'B_1_to_30_days' THEN pd.OS_Principal_LC ELSE 0 END) AS B_1_to_30_days,
        SUM(CASE WHEN pd.Arrears_Category = 'C_31_to_60_days' THEN pd.OS_Principal_LC ELSE 0 END) AS C_31_to_60_days,
        SUM(CASE WHEN pd.Arrears_Category = 'D_61_to_90_days' THEN pd.OS_Principal_LC ELSE 0 END) AS D_61_to_90_days,
        SUM(CASE WHEN pd.Arrears_Category = 'E_91_to_120_days' THEN pd.OS_Principal_LC ELSE 0 END) AS E_91_to_120_days,
        SUM(CASE WHEN pd.Arrears_Category = 'F_121_to_180_days' THEN pd.OS_Principal_LC ELSE 0 END) AS F_121_to_180_days,
        SUM(CASE WHEN pd.Arrears_Category = 'G_181_to_270_days' THEN pd.OS_Principal_LC ELSE 0 END) AS G_181_to_270_days
    FROM MigrationBase mb
    RIGHT JOIN Processed_Data pd ON mb.Loan_ID = pd.Loan_ID
    GROUP BY mb.CategoryBase
    ORDER BY mb.CategoryBase;
""")

#check the outputs
con.execute("SELECT * FROM Migration_NewBuckets").df()

,Arrears_Category,A_Current,B_1_to_30_days,C_31_to_60_days,D_61_to_90_days,E_91_to_120_days,F_121_to_180_days,G_181_to_270_days
0,A_Current,2.394335e+07,250880.64,99763.76,0.00,0.00,0.00,0.00
1,B_1_to_30_days,1.630967e+04,3171.68,610.05,764.75,36352.57,0.00,0.00
2,C_31_to_60_days,8.178000e+01,0.00,0.00,50.49,8581.88,0.00,0.00
3,D_61_to_90_days,1.100741e+04,3270.84,182.84,0.00,3442.68,107898.53,0.00
4,E_91_to_120_days,1.167900e+03,0.00,0.00,0.00,1890.93,28703.55,0.00
5,F_121_to_180_days,3.037110e+03,65.72,67.00,0.00,0.00,69.96,10445.83
6,G_181_to_270_days,0.000000e+00,0.00,0.00,0.00,0.00,0.00,36465.68
7,None,7.397822e+06,0.00,0.00,0.00,0.00,0.00,0.00


### 3. Determine volumes in reference buckets which are written-off as of the new period

In [11]:
#Step 3: Migration_WriteOffs

# Delete table if already exists
con.execute("DROP TABLE IF EXISTS Migration_WriteOffs;")

# We set a cut-off limit to exclude loans that were written off post the period of the new data.
cutoff_date = '2021-10-31'

# This code links the MigrationBase with "Woff_Data" 
# and generates a breakdown of written-off loans by their original bucket category in MigrationBase.
con.execute(f"""
    CREATE TABLE Migration_WriteOffs AS
    SELECT 
        mb.CategoryBase,
        SUM(COALESCE(wd.WOFF_AMOUNT_PRINCIPLE, 0) + COALESCE(wd.WOFF_AMOUNT_INTEREST, 0)) AS Write_Off
    FROM MigrationBase mb
    LEFT JOIN Woff_Data wd ON mb.Loan_ID = wd.Loan_ID
    WHERE wd.WOFF_Date IS NOT NULL 
      AND STRPTIME(wd.WOFF_Date, '%d/%m/%Y') <= DATE '{cutoff_date}'
    GROUP BY mb.CategoryBase
    ORDER BY mb.CategoryBase;
""")

#check the outputs
con.execute("SELECT * FROM Migration_WriteOffs").df()

,CategoryBase,Write_Off
0,F_121_to_180_days,72669.89
1,G_181_to_270_days,21313.64


### 4. Combine the outcomes from the previous tables

In [12]:
# Step 4: Migration_Final_Vol_ratios

# Delete table if already exists
con.execute("DROP TABLE IF EXISTS Migration_Final_Vol_ratios;")

# This code uses the outputs of the preiviously generated tables and link them to generate the final migration analysis. 
con.execute("""
    CREATE TABLE Migration_Final_Vol_ratios AS
    SELECT 
        nb.Arrears_Category,
        bb.OS_Principal,
        nb.A_Current / bb.OS_Principal AS "Current",
        nb.B_1_to_30_days / bb.OS_Principal AS "1_to_30_days",
        nb.C_31_to_60_days / bb.OS_Principal AS "31_to_60_days",
        nb.D_61_to_90_days / bb.OS_Principal AS "61_to_90_days",
        nb.E_91_to_120_days / bb.OS_Principal AS "91_to_120_days",
        nb.F_121_to_180_days / bb.OS_Principal AS "121_to_180_days",
        nb.G_181_to_270_days / bb.OS_Principal AS "181_to_270_days",
        wo.Write_Off / bb.OS_Principal AS "Write_off"
    FROM Migration_NewBuckets nb
    LEFT JOIN Migration_WriteOffs wo ON nb.Arrears_Category = wo.CategoryBase
    RIGHT JOIN Migration_BaseBuckets bb ON nb.Arrears_Category = bb.Arrears_Category;
""")


Here we check the outcomes of the final migration analysis with additional formatting. Note that the maximum migration limit is two buckets forward since the difference between the datasets is only two months.

In [13]:
#Examine final migration analysis
final_df = con.execute("SELECT * FROM Migration_Final_Vol_ratios").df().sort_values('Arrears_Category')

#Identifying all ratio columns (excluding OS_Principal and Arrears_Category)
ratio_cols = final_df.columns.difference(['Arrears_Category', 'OS_Principal'])

#Converting proportions to percentages
final_df[ratio_cols] = final_df[ratio_cols].applymap(lambda x: f"{x*100:.2f}%") 

#Round to two decimals
final_df[ratio_cols] = final_df[ratio_cols].round(2)


final_df

C:\Users\Mohammad\AppData\Local\Temp\ipykernel_3808\2263244796.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df[ratio_cols] = final_df[ratio_cols].applymap(lambda x: f"{x*100:.2f}%")


,Arrears_Category,OS_Principal,Current,1_to_30_days,31_to_60_days,61_to_90_days,91_to_120_days,121_to_180_days,181_to_270_days,Write_off
2,A_Current,30813690.80,77.70%,0.81%,0.32%,0.00%,0.00%,0.00%,0.00%,nan%
3,B_1_to_30_days,71180.90,22.91%,4.46%,0.86%,1.07%,51.07%,0.00%,0.00%,nan%
4,C_31_to_60_days,9492.70,0.86%,0.00%,0.00%,0.53%,90.41%,0.00%,0.00%,nan%
5,D_61_to_90_days,142050.73,7.75%,2.30%,0.13%,0.00%,2.42%,75.96%,0.00%,nan%
6,E_91_to_120_days,35354.34,3.30%,0.00%,0.00%,0.00%,5.35%,81.19%,0.00%,nan%
0,F_121_to_180_days,197256.36,1.54%,0.03%,0.03%,0.00%,0.00%,0.04%,5.30%,36.84%
1,G_181_to_270_days,56960.26,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,64.02%,37.42%
